## Modeling을 위한

In [1]:
import warnings
warnings.filterwarnings(action='ignore')
from functools import partial
from shapely import ops
import pyproj
import json, math
import matplotlib.pyplot as plt
from folium import plugins
import re
from matplotlib import font_manager, rc, rcParams
# def set_korea_font():
#     font_name = font_manager.FontProperties(fname="/System/Library/Fonts/Supplemental/AppleGothic.ttf").get_name()
#     rc('font', family=font_name)
#     rcParams.update({'font.size': 11})
#     rcParams['axes.unicode_minus'] = False  
# set_korea_font()
from shapely.geometry import Point as shapely_Point
from geopy.distance import great_circle as distance
from geopy.point import Point as Point
from math import sin, cos, atan2, sqrt, degrees, radians, pi

from geopy.distance import geodesic
import folium
from folium import plugins
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely
import os
import datetime

from tqdm import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

## 1) 25m 당 각 도로의 특징 데이터 만들기

In [2]:
os.chdir('../data')

# 실제 거리를 구하기 위해 사용되는 reproject 함수
def reproject(geom, from_proj=None, to_proj=None):
    tfm = partial(pyproj.transform, pyproj.Proj(init=from_proj), pyproj.Proj(init=to_proj))
    return ops.transform(tfm, geom)

In [3]:
os.getcwd()

'/Users/jominju/Desktop/대학교/contest/Car_Accident/최종/data'

### (1) 필요한 데이터 가져오기

In [4]:
# 필요한 데이터

overspeed_cam_count_ = pd.read_csv('overspeed_cam.csv')
floating_pop_count_ = pd.read_csv('floating_pop.csv')
bump_count_ = pd.read_csv('bump.csv')
parking_count_ = pd.read_csv('parking.csv')
parking_cctv_count_ = [parking_count_, pd.read_csv('parking_cctv.csv')]
car_count_ = gpd.read_file("3.오산시_차량등록현황_격자.geojson", encoding = "utf-8")
child_count_ = gpd.read_file("child_count.geojson", encoding = "utf-8")
child_rate_count_ = gpd.read_file("child_rate.geojson", encoding = "utf-8")
elem_kinder_count_ = gpd.read_file("elem_kinder_count.csv", encoding = "utf-8")
isSchoolZone_count_ = pd.read_csv("9.오산시_어린이보호구역(re).csv", encoding = "utf-8")
numberSchoolZone_count_ = isSchoolZone_count_
accident_count_ = pd.read_csv("accident_count.csv")
chaos1_nearby_count_ = [gpd.read_file('23.오산시_상세도로망_LV6.geojson'),
                 pd.read_csv('chaos1.csv')]
width_nearby_count_ = gpd.read_file('23.오산시_상세도로망_LV6.geojson')
cross_road_nearby_count_ = width_nearby_count_
num_cram_school_count_ = pd.read_csv("학원.csv")
shortest_cross_count_ = gpd.read_file("횡단보도.json",encoding="utf-8")
shortest_traffic_signal_count_ = gpd.read_file('19.오산시_신호등.geojson',encoding="utf-8")
shortest_sidewalk_count_ =  gpd.read_file("인도.json",encoding="utf-8")

In [5]:
# Dictionary로 정리

data_set = {
'overspeed_cam_count': overspeed_cam_count_,
'floating_pop_count':floating_pop_count_,
'bump_count':bump_count_,
'parking_count':parking_count_,
'parking_cctv_count':parking_cctv_count_,
'car_count':car_count_,
'child_count':child_count_,
'child_rate_count':child_rate_count_,
'elem_kinder_count':elem_kinder_count_,
'isSchoolZone_count':isSchoolZone_count_,
'numberSchoolZone_count':numberSchoolZone_count_,
'accident_count':accident_count_,
'chaos1_nearby_count':chaos1_nearby_count_,
'width_nearby_count':width_nearby_count_,
'cross_road_nearby_count':cross_road_nearby_count_,
'num_cram_school_count':num_cram_school_count_,
'shortest_cross_count':shortest_cross_count_,
'shortest_traffic_signal_count':shortest_traffic_signal_count_,
'shortest_sidewalk_count':shortest_sidewalk_count_,
'barrier_nearby_count':width_nearby_count_,
}

# 어린이 교통사고 정보만 필요하므로 나이가 12세 이하인 사고 정보만 가져온다

data_set['accident_count'].loc[:,'가해운전자 연령'] = data_set['accident_count'].loc[:,'가해운전자 연령'].apply(lambda x: int(x[:-1]) if x[:-1].isnumeric() else 999)
data_set['accident_count'].loc[:,'피해운전자 연령'] = data_set['accident_count'].loc[:,'피해운전자 연령'].apply(lambda x: int(x[:-1]) if str(x)[:-1].isnumeric() else 999)

mask1 = data_set['accident_count'].loc[:,'가해운전자 연령'] <= 12
mask2 = data_set['accident_count'].loc[:,'피해운전자 연령'] <= 12

data_set['accident_count'] = data_set['accident_count'].loc[mask1|mask2]
data_set['accident_count'].index = range(len(data_set['accident_count']))

In [6]:
# tuple이 str으로 저장되는 문제를 해결하기 위한 함수
def _str_to_tuple(string):
  """
  input: multilinestring object
  output: arr_cords; lon, lat
  ex) multiline_to_arr_cords(road_gdf.loc[0].geometry)
  """
  return re.findall(r'\d+\.\d+',string)

### (2) 데이터를 거리별로 가공하기 위한 함수
- 여기에 해당하는 모든 함수는 parameter를 모두 (lat, lon, radi, refer_data)로 통일하였다. idx parameter는 필요에 따라 넣었다.
- lat, lon : 해당 도로의 lat, lon (input 도로)
- radi : 해당 도로의 반경 (ex. radi = 50 -> 해당 도로 좌표 50m 반경의 데이터만 취급)
- refer_data : 해당 함수가 취급하는 데이터
- idx
   - if False (default) -> 해당 refer_data의 모든 데이터를 사용
   - else : idx에 해당하는 refer_data만 사용
   
   
   
- 함수 목록
    - 과속단속 카메라 개수
    - 유동인구 평균
    - 법규위반
    - 과속방지턱
    - 불법주정차 단속 건수
    - cctv 1개 당 평균 불법주정차 단속 건수
    - 차량등록현황
    - 어린이 거주인구
    - 어린이 거주 비율
    - 주변 학교/어린이집 개수
    - 스쿨존 여부
    - 포함되는 스쿨존 수
    - 사고 수
    - 혼잡빈도
    - 도로폭
    - 사거리 여부
    - 주변 어린이 대상 학원/교습소 수
    - 횡단보도와의 최소 거리
    - 신호등과의 최소 거리
    - 인도와의 최소 거리
    - 주변 중앙 분리대 개수

In [7]:
# "(2) 데이터를 거리별로 가공하기 위한 함수"를 사용하기 위해 geometry와 좌표 간의 실제 거리를 구하는 함수

#from https://medium.com/analytics-vidhya/calculating-distances-from-points-to-polygon-borders-in-python-a-paris-example-3b597e1ea291
def midpoint(a, b):
    a_lat, a_lon = radians(a.latitude), radians(a.longitude)
    b_lat, b_lon = radians(b.latitude), radians(b.longitude)
    delta_lon = b_lon - a_lon
    B_x = cos(b_lat) * cos(delta_lon)
    B_y = cos(b_lat) * sin(delta_lon)
    mid_lat = atan2(
        sin(a_lat) + sin(b_lat),
        sqrt(((cos(a_lat) + B_x) ** 2 + B_y ** 2))
    )
    mid_lon = a_lon + atan2(B_y, cos(a_lat) + B_x)
    # Normalise
    mid_lon = (mid_lon + 3 * pi) % (2 * pi) - pi

    return Point(latitude=degrees(mid_lat), longitude=degrees(mid_lon))

def get_line_midpoint(line):
    a = Point(line[0])
    b = Point(line[1])

    return midpoint(a,b)


def calculate_dist_to_line(line_a_lat, line_a_lng, line_b_lat, line_b_lng, point_object):
    a = Point(latitude=line_a_lat, longitude=line_a_lng)
    b = Point(latitude=line_b_lat, longitude=line_b_lng)
    dist = distance(midpoint(a, b), point_object)
    return dist


def get_min_distance_to_arr(arr_coords, point_object, unit='m'):
    min_dist = 999999
    line=[]
    for i, _ in enumerate(arr_coords):
        if i + 1 < len(arr_coords):
            dist = calculate_dist_to_line(
                line_a_lat=arr_coords[i][1],
                line_a_lng=arr_coords[i][0],
                line_b_lat=arr_coords[i + 1][1],
                line_b_lng=arr_coords[i + 1][0],
                point_object=point_object
            )
            if dist < min_dist:
                min_dist = dist
                line = [(arr_coords[i][1], arr_coords[i][0]), (arr_coords[i + 1][1], arr_coords[i + 1][0])]
        else:
            dist = calculate_dist_to_line(
                line_a_lat=arr_coords[i][1],
                line_a_lng=arr_coords[i][0],
                line_b_lat=arr_coords[0][1],
                line_b_lng=arr_coords[0][0],
                point_object=point_object
            )
            if dist < min_dist:
                min_dist = dist
                line = [(arr_coords[i][1], arr_coords[i][0]), (arr_coords[0][1], arr_coords[0][0])]

    if unit == 'm':
        return min_dist.m, line
    elif unit == 'km':
        return min_dist.km, line
    else:
        return min_dist, line
    
#additional interrial function
def _multiline_to_arr_cords(mls):
  """
  input: multilinestring object
  output: arr_cords; lon, lat
  ex) multiline_to_arr_cords(road_gdf.loc[0].geometry)
  """
  arr_cords = []
  lon_lat = re.findall(r'\d+\.\d+',str(mls))
  for i in range(0,len(lon_lat),2):
    arr_cords.append((float(lon_lat[i]),float(lon_lat[i+1])))
  return arr_cords

def _shPoint_to_list(shpoint):
  c1, c2 = re.findall(r'\d+\.\d+',str(shpoint))
  if c1 < c2:
    lon, lat = float(c1), float(c2)
  else:
    lat, lon = float(c1), float(c2)
  return lat, lon

def _list_to_gpPoint(mylist):
  return Point(latitude=mylist[0], longitude=mylist[1])

def _shPoint_to_gpPoint(shpoint):
  mylist = _shPoint_to_list(shpoint)
  return Point(latitude=mylist[0], longitude=mylist[1])

In [8]:
def road_nearby(lat, lon, radi, data):
  """
  input: origin lat lon, radi(m), data(shapely multilinestring)
  change in type: shapely(Euclidian) -> list -> geopy(Great Circle)
  output: following roads
  """
  road_gdf = data
  dist_array = road_gdf.geometry.apply(lambda x: get_min_distance_to_arr(_multiline_to_arr_cords(x), _shPoint_to_gpPoint((lat, lon)), unit='m')[0])
  result = road_gdf.loc[dist_array < radi,:]
  while (result.size == 0):
    radi += 5
    result = road_gdf.loc[dist_array < radi,:]
  return result

### a. 과속단속 카메라 개수

In [9]:
def overspeed_cam_count(lat, lon, radi, refer_data, idx=False):
  """
  input: origin lat lon, radi(m)
  output: following point count
  optional: (idx of point, count)
  """
  if (type(refer_data['geometry'][0]) == str):
    refer_data['geometry'] = refer_data['geometry'].apply(_str_to_tuple)
  #apply
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon),x).meters <= radi)
  if idx:
    return refer_data[bool_mask].index, sum(bool_mask)
  else:
    return sum(bool_mask)

### b. 유동인구 평균

In [10]:
def floating_pop_count(lat, lon, radi, refer_data, idx=False):
  """
  input: origin lat lon, radi(km)
  output: following point count
  optional: (idx of point, count)
  """
  if (type(refer_data['geometry'][0]) == str):
    refer_data['geometry'] = refer_data['geometry'].apply(_str_to_tuple)
  #apply
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon), x).m <= radi)
  if idx:
    return refer_data[bool_mask].index, refer_data['mean'][bool_mask].mean()
  else:
    return refer_data['mean'][bool_mask].mean()

### c. 과속방지턱

In [11]:
def bump_count(lat, lon, radi, refer_data, idx = False):
    if (type(refer_data['geometry'][0]) == str):
        refer_data['geometry'] = refer_data['geometry'].apply(_str_to_tuple)
    dist = refer_data[['index','geometry']].apply(lambda x: (geodesic((lat, lon), x[1]).m, x[0]), axis = 1)
    dist = min(dist, key = lambda x :x[0])
                                        
    if idx:
        return dist
    else:
        return dist[0]

### d. 불법주정차

In [12]:
def parking_count(lat, lon, radi, refer_data, idx=False):
  """
  input: origin lat lon, radi(km)
  output: following point count
  optional: (idx of point, count)
  """
  if (type(refer_data['geometry'][0]) == str):
    refer_data['geometry'] = refer_data['geometry'].apply(_str_to_tuple)
  #apply
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon),x).meters <= radi)
  if idx:
    return refer_data[bool_mask].index, sum(bool_mask)
  else:
    return sum(bool_mask)

### e. cctv 1개 당 평균 불법주정차 단속 건수

In [13]:
def parking_cctv_count(lat, lon, radi, refer_data, idx=False):
  """
  cctv 1개당 평균 불법주정차 단속 건수
  input: origin lat lon, radi(km)
  output: following point count
  optional: (idx of point, count)
  """
  refer_data1, refer_data2 = refer_data[0], refer_data[1]
  if (type(refer_data1['geometry'][0]) == str):
    refer_data1['geometry'] = refer_data1['geometry'].apply(_str_to_tuple)
  if (type(refer_data2['geometry'][0]) == str):
    refer_data2['geometry'] = refer_data2['geometry'].apply(_str_to_tuple)
    
    
  #apply
  bool_mask1 = refer_data1['geometry'].apply(lambda x: geodesic((lat, lon),x).meters <= radi)
  bool_mask2 = refer_data2['geometry'].apply(lambda x: geodesic((lat, lon),x).meters <= radi)

  try:    
    result = sum(bool_mask1)/sum(bool_mask2)
  except:
    result = np.nan
  if idx:
    return refer_data1[bool_mask1].index, refer_data2[bool_mask2].index, result
  else:
    return result

### f. 차량 등록 현황

In [14]:
def car_count(lat, lon, radi, refer_data):
    """
    default unit = 'm'
    """   
    distance_arr = refer_data.geometry.apply(lambda x : get_min_distance_to_arr
                    (_multiline_to_arr_cords(x),
                     _shPoint_to_gpPoint((lat, lon)), unit = 'm')[0])
    sum_count = refer_data[distance_arr <= radi]["car_cnt"].max()
    return sum_count

### g. 어린이 거주 인구

In [15]:
def child_count(lat, lon, radi, refer_data):
    refer_data.val[refer_data.val.isna()] = 0
    distance_arr = refer_data.geometry.apply(lambda x : get_min_distance_to_arr
                    (_multiline_to_arr_cords(x),
                     _shPoint_to_gpPoint((lat, lon)), unit = 'm')[0])
    sum_count = refer_data[distance_arr <= radi]["val"].max()
    return sum_count

### h. 어린이 거주 비율

In [16]:
def child_rate_count(lat, lon, radi, refer_data):
    distance_arr = refer_data.geometry.apply(lambda x : get_min_distance_to_arr
                    (_multiline_to_arr_cords(x),
                     _shPoint_to_gpPoint((lat, lon)), unit = 'm')[0])
    selected_data = refer_data[distance_arr <= radi]
    child_rate = sum(selected_data["val"])/sum(selected_data["total"])
    return child_rate

### i. 주변 학교 / 어린이집 개수

In [17]:
def elem_kinder_count(lat, lon, radi, refer_data, typ = 'all'):
    """
    typ = 'all', '유치원과 어린이집', '초등학교'
    """
    if typ != 'all':
        refer_data = refer_data.loc[refer_data.시설구분==typ]
    data_loc = pd.Series(list(zip(refer_data["시설위치_위도"], refer_data["시설위치_경도"])))
    count_sum = data_loc.apply(lambda x: geodesic((lat, lon),x).m <= radi).sum()
    return count_sum

### j. 스쿨존 여부

In [18]:
def isSchoolZone_count(lat, lon, radi, refer_data): # radi = 300 이지만 순서 통일을 위해 default값 지정 x
    data_loc = pd.Series(list(zip(refer_data["보호구역_위도"], refer_data["보호구역_경도"])))
    count_sum = data_loc.apply(lambda x: geodesic((lat, lon),x).m <= 300).sum()
    return count_sum > 0

### k. 포함되는 스쿨존 수

In [19]:
def numberSchoolZone_count(lat, lon, radi, refer_data): # radi = 300 이지만 순서 통일을 위해 default값 지정 x
    data_loc = pd.Series(list(zip(refer_data["보호구역_위도"], refer_data["보호구역_경도"])))
    count_sum = data_loc.apply(lambda x: geodesic((lat, lon),x).m <= 300).sum()
    return count_sum

### l. 사고 수

In [20]:
def accident_count(lat, lon, radi, refer_data, idx=False):
  """
  input: origin lat lon, radi(m), data(accidents record)
  output: following point count
  optional: (idx of point, count)
  """

  if (type(refer_data['geometry'][0]) == str):
    refer_data['geometry'] = refer_data['geometry'].apply(_str_to_tuple)
  #apply
  bool_mask = refer_data['geometry'].apply(lambda x: geodesic((lat, lon),x).m <= radi)
  if idx:
    return refer_data[bool_mask].index, sum(bool_mask)
  else:
    return sum(bool_mask)

### m. 혼잡빈도

In [21]:
def chaos1_nearby_count(lat, lon, radi, refer_data):
  """
  input: origin lat lon, radi(m), road_data(shapely multilinestring), chaos1_data
  output: following road's chaos1 by different times or lanes
  
  #test_set
  lat, lon = (37.14875860564099, 127.0773701360968)
  radi = 1
  chaos1_nearby(lat, lon, radi, road_gdf, chaos1_gdf)[11]
  #link_id 끝자리 01(상행) 02(하행) 순으로 나옴
  평일 주말 총 평균값
   """
  road_data, chaos1_data = refer_data[0], refer_data[1]
  roads = road_nearby(lat, lon, radi, road_data)
  result = roads.link_id.apply(lambda x: chaos1_data.loc[(chaos1_data.loc[:,'link_id']//100)==int(x),['time_range','chaos1']].mean()[0])
  return result.mean()

### n. 도로 폭

In [22]:
def width_nearby_count(lat, lon, radi, road_data):
  roads = road_nearby(lat, lon, radi, road_data)
  return roads.width.apply(int).mean()

### o. 사거리 여부

In [23]:
def cross_road_nearby_count(lat, lon, radi, road_data):
  """
  4	교차로 통로
  32	복합교차로
  64	로타리
  128	회전교차로

  output = # of cross roads
  """
  cross_roads = [4,32,64,128]
  roads = road_nearby(lat, lon, radi, road_data)
  result = roads.loc[roads.link_type.apply(lambda x: int(x) in cross_roads)]
  return len(result)

### p. 주변 학교 / 교습소 수

In [24]:
def num_cram_school_count(lat, lon, radi, refer_data):
    cram_school = pd.Series(list(zip(refer_data["시설위치_위도"], refer_data["시설위치_경도"])))
    return cram_school.apply(lambda x: geodesic((lat, lon),x).m <= radi).sum()

### q. 횡단보도와의 최소 거리

In [25]:
#reproject 후 polygon 그대로
def shortest_cross_count(lat, lon, radi, refer_data, idx = False):
    polygon1 = reproject(shapely_Point(lon,lat), 'EPSG:4326', 'EPSG:26944')
        
    result = refer_data[["OBJECTID","geometry"]].apply(lambda x : (x[1].distance(polygon1), x[0]), axis = 1)
    result = min(result, key = lambda x :x[0])
    if idx:
        return result
    return result[0]

### r. 신호등과의 최소 거리

In [26]:
def shortest_traffic_signal_count(lat, lon, radi, refer_data, idx = False):
    result = refer_data[["OBJECTID","geometry"]].apply(lambda x : (geodesic((lat, lon),(x[1].y,x[1].x)).m, x[0]), axis = 1)
    result = min(result, key = lambda x :x[0])
    if idx:
        return result
    return result[0]

### s. 인도와의 최소 거리

In [27]:
def shortest_sidewalk_count(lat, lon, radi, refer_data):
    polygon1 = reproject(shapely_Point(lon,lat), 'EPSG:4326', 'EPSG:26944')
    result = refer_data["geometry"].apply(lambda x : x.distance(polygon1))
    result = min(result)
    return result

### t. 주변 중앙 분리대 수

In [28]:
def barrier_nearby_count(lat, lon, radi, road_data):
  roads = road_nearby(lat, lon, radi, road_data)
  return int(~(roads.barrier.apply(int).max() == 0))

### (3) (2)의 함수를 사용해서 feature 만들기

In [29]:
# 함수를 dictionary로 정리

function_dict = {
    "overspeed_cam_count" : overspeed_cam_count,
    "floating_pop_count" : floating_pop_count,
    "bump_count" : bump_count,
    "parking_count" : parking_count,
    "parking_cctv_count" : parking_cctv_count,
    "car_count" : car_count,
    "child_count" : child_count,
    "child_rate_count" : child_rate_count,
    "elem_kinder_count" : elem_kinder_count,
    "isSchoolZone_count" : isSchoolZone_count,
    "numberSchoolZone_count" : numberSchoolZone_count,
    "accident_count" : accident_count,
    "chaos1_nearby_count" : chaos1_nearby_count,
    "width_nearby_count" : width_nearby_count,
    "cross_road_nearby_count" : cross_road_nearby_count,
    "num_cram_school_count" : num_cram_school_count,
    "shortest_cross_count" : shortest_cross_count,
    "shortest_traffic_signal_count" : shortest_traffic_signal_count,
    "shortest_sidewalk_count" : shortest_sidewalk_count,
    "barrier_nearby_count" : barrier_nearby_count}

In [30]:
# 각 함수별로 사용되는 반경이 다르기 때문에
# 함수별로 사용되는 반경을 dictionary의 value로 저장

radi_dict = {
    'overspeed_cam_count' : [100],
    'floating_pop_count' : [50],
    'bump_count' : [0], # [12.5 + 15, 25 + 15],
   'parking_count' : [12.5, 25],
    'car_count' : [1000], # max값
    'child_count' : [1000], # max값
    'elem_kinder_count' : [400], 
    'numberSchoolZone_count' : [400],
    'accident_count' : [12.5, 25],
    'chaos1_nearby_count' : [12.5, 25, 50],
    'width_nearby_count' : [12.5, 25],
    'cross_road_nearby_count' : [12.5, 25],
    'num_cram_school_count' : [400],
    'shortest_cross_count' : [0],
    'shortest_traffic_signal_count' : [0],
    'shortest_sidewalk_count' : [0],
    'barrier_nearby_count' : [12.5, 25]}

데이터 제작  
시간이 오래 걸리므로 미리 만든데이터 사용 권장

In [31]:
df = pd.read_csv("learn_point25.csv")
values = [df]
for key in list(radi_dict.keys()):
    for radi in radi_dict[key]:
        print(key + str(radi))
        tmp = df.progress_apply(lambda x: function_dict[key](x[1], x[0], radi, data_set[key]),axis = 1)
        if len(tmp.shape) == 1:
            tmp.name = key + str(radi)
        else:
            tmp.columns = [key + str(radi) +"_" + str(i) for i in range(tmp.shape[1])]
        values.append(tmp)

pd.concat(values,axis=1).to_csv("0_8856.csv",index = False)

overspeed_cam_count100



floating_pop_count50



bump_count0



parking_count12.5



parking_count25


...


shortest_cross_count0



shortest_traffic_signal_count0



shortest_sidewalk_count0



barrier_nearby_count12.5



barrier_nearby_count25
